In [ ]:
import os
import glob
from ipywidgets import Dropdown, Label, FloatText, VBox, HBox

In [ ]:
# Show available test cases
testcases = [(os.path.basename(path)[:-5], path) for path in glob.glob('../*.yaml')]
dropdown = Dropdown(options=testcases, description='Test case:')
display(dropdown)

In [ ]:
# Initialize the test case (this reads fabm.yaml)
import pyfabm
model = pyfabm.Model(dropdown.value)

In [ ]:
# Present configurable environmental conditions
default_environment = {
    'temperature': 15,
    'practical_salinity': 35,
    'surface_downwelling_photosynthetic_radiative_flux': 50,
    'downwelling_photosynthetic_radiative_flux': 10,
    'bottom_depth': 10,
    'density': 1020
}
labels, inputs = [], []
for variable in model.dependencies:
    labels.append(Label('%s:' % variable.long_name))
    inputs.append(FloatText(value=default_environment.get(variable.name, 0.), layout={'width': '7em'}))
display(HBox((VBox(labels), VBox(inputs))))

In [ ]:
# Transfer environmental conditions to model
for variable, widget in zip(model.dependencies, inputs):
    variable.value = widget.value
assert model.start(), 'Model failed to start: %s' % pyfabm.getError()

In [ ]:
# Time integrate
import numpy
import scipy.integrate

# Time derivative
def dy(y, t0):
    model.state[:] = y
    return model.getRates(t0)

# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
t = numpy.arange(0, 200., 1)
y = scipy.integrate.odeint(dy, model.state, t*86400)

In [ ]:
# Plot results
%matplotlib notebook
from matplotlib import pyplot
pyplot.plot(t, y)
pyplot.legend([variable.long_path for variable in model.state_variables])
pyplot.show()

In [ ]:
model.state